In [1]:
pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 37.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

In [3]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [4]:
def create_bucket(bucket):
    import logging

    try:
        s3.create_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError as e:
        logging.error(e)
        return 'Bucket ' + bucket + ' could not be created.'
    return 'Created or already exists ' + bucket + ' bucket.'

In [5]:
create_bucket('nyctlc-cs653-5181')

'Created or already exists nyctlc-cs653-5181 bucket.'

In [6]:
def list_buckets(match=''):
    response = s3.list_buckets()
    if match:
        print(f'Existing buckets containing "{match}" string:')
    else:
        print('All existing buckets:')
    for bucket in response['Buckets']:
        if match:
            if match in bucket["Name"]:
                print(f'  {bucket["Name"]}')

In [7]:
list_buckets(match='open')

Existing buckets containing "open" string:


In [8]:
def list_bucket_contents(bucket, match='', size_mb=0):
    bucket_resource = s3_resource.Bucket(bucket)
    total_size_gb = 0
    total_files = 0
    match_size_gb = 0
    match_files = 0
    for key in bucket_resource.objects.all():
        key_size_mb = key.size/1024/1024
        total_size_gb += key_size_mb
        total_files += 1
        list_check = False
        if not match:
            list_check = True
        elif match in key.key:
            list_check = True
        if list_check and not size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')
        elif list_check and key_size_mb <= size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')

    if match:
        print(f'Matched file size is {match_size_gb/1024:3.1f}GB with {match_files} files')            
    
    print(f'Bucket {bucket} total size is {total_size_gb/1024:3.1f}GB with {total_files} files')

In [9]:
list_bucket_contents(bucket='nyc-tlc', match='2017', size_mb=250)

csv_backup/green_tripdata_2017-01.csv ( 91MB)
csv_backup/green_tripdata_2017-02.csv ( 87MB)
csv_backup/green_tripdata_2017-03.csv ( 99MB)
csv_backup/green_tripdata_2017-04.csv ( 92MB)
csv_backup/green_tripdata_2017-05.csv ( 91MB)
csv_backup/green_tripdata_2017-06.csv ( 83MB)
csv_backup/green_tripdata_2017-07.csv ( 78MB)
csv_backup/green_tripdata_2017-08.csv ( 74MB)
csv_backup/green_tripdata_2017-09.csv ( 75MB)
csv_backup/green_tripdata_2017-10.csv ( 79MB)
csv_backup/green_tripdata_2017-11.csv ( 75MB)
csv_backup/green_tripdata_2017-12.csv ( 77MB)
trip data/fhv_tripdata_2017-01.parquet ( 55MB)
trip data/fhv_tripdata_2017-02.parquet ( 54MB)
trip data/fhv_tripdata_2017-03.parquet ( 64MB)
trip data/fhv_tripdata_2017-04.parquet ( 60MB)
trip data/fhv_tripdata_2017-05.parquet ( 64MB)
trip data/fhv_tripdata_2017-06.parquet (134MB)
trip data/fhv_tripdata_2017-07.parquet (137MB)
trip data/fhv_tripdata_2017-08.parquet (139MB)
trip data/fhv_tripdata_2017-09.parquet (144MB)
trip data/fhv_tripdata_20

In [10]:
def preview(bucket, key):
    data_source = {
            'Bucket': bucket,
            'Key': key
        }
    # Generate the URL to get Key from Bucket
    url = s3.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )

    data = pd.read_parquet(url, engine='pyarrow')
    return data

In [11]:
df = preview(bucket='nyc-tlc', key=f'trip data/yellow_tripdata_2017-01.parquet')
df.head(6)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2017-01-01 00:32:05,2017-01-01 00:37:48,1,1.2,1,N,140,236,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8,None,None
1,1,2017-01-01 00:43:25,2017-01-01 00:47:42,2,0.7,1,N,237,140,2,5.0,0.5,0.5,0.0,0.0,0.3,6.3,None,None
2,1,2017-01-01 00:49:10,2017-01-01 00:53:53,2,0.8,1,N,140,237,2,5.5,0.5,0.5,0.0,0.0,0.3,6.8,None,None
3,1,2017-01-01 00:36:42,2017-01-01 00:41:09,1,1.1,1,N,41,42,2,6.0,0.5,0.5,0.0,0.0,0.3,7.3,None,None
4,1,2017-01-01 00:07:41,2017-01-01 00:18:16,1,3.0,1,N,48,263,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3,None,None
5,1,2017-01-01 00:20:52,2017-01-01 00:24:59,2,0.7,1,N,236,262,2,5.0,0.5,0.5,0.0,0.0,0.3,6.3,None,None


In [12]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2017-01-01 00:32:05,2017-01-01 00:37:48,1,1.2,1,N,140,236,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8,None,None
1,1,2017-01-01 00:43:25,2017-01-01 00:47:42,2,0.7,1,N,237,140,2,5.0,0.5,0.5,0.0,0.0,0.3,6.3,None,None
2,1,2017-01-01 00:49:10,2017-01-01 00:53:53,2,0.8,1,N,140,237,2,5.5,0.5,0.5,0.0,0.0,0.3,6.8,None,None
3,1,2017-01-01 00:36:42,2017-01-01 00:41:09,1,1.1,1,N,41,42,2,6.0,0.5,0.5,0.0,0.0,0.3,7.3,None,None
4,1,2017-01-01 00:07:41,2017-01-01 00:18:16,1,3.0,1,N,48,263,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3,None,None


In [13]:
df.shape

(9710820, 19)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9710820 entries, 0 to 9710819
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        int64         
 4   trip_distance          float64       
 5   RatecodeID             int64         
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   object        
 18  airport_fee           

In [15]:
df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06,9.710820e+06
mean,1.547111e+00,1.629222e+00,2.813879e+00,1.039580e+00,1.641076e+02,1.617635e+02,1.337539e+00,1.237414e+01,3.234837e-01,4.975229e-01,1.751259e+00,2.764088e-01,2.996798e-01,1.552662e+01
std,4.977756e-01,1.272268e+00,3.611637e+00,5.058933e-01,6.665001e+01,7.067210e+01,4.913687e-01,2.652220e+02,4.425521e-01,4.881261e-02,2.571219e+00,1.638808e+00,1.342100e-02,2.653384e+02
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-3.500000e+02,-5.520000e+01,-5.000000e-01,-4.100000e+01,-1.500000e+01,-3.000000e-01,-3.503000e+02
25%,1.000000e+00,1.000000e+00,9.500000e-01,1.000000e+00,1.140000e+02,1.070000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.300000e+00
50%,2.000000e+00,1.000000e+00,1.600000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,0.000000e+00,5.000000e-01,1.300000e+00,0.000000e+00,3.000000e-01,1.130000e+01
75%,2.000000e+00,2.000000e+00,2.900000e+00,1.000000e+00,2.330000e+02,2.340000e+02,2.000000e+00,1.350000e+01,5.000000e-01,5.000000e-01,2.260000e+00,0.000000e+00,3.000000e-01,1.673000e+01
max,2.000000e+00,9.000000e+00,2.647100e+02,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,6.259008e+05,5.554000e+01,5.650000e+01,9.999900e+02,9.110800e+02,3.000000e-01,6.259016e+05


In [16]:
def key_exists(bucket, key):
    try:
        s3_resource.Object(bucket, key).load()
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            # The key does not exist.
            return(False)
        else:
            # Something else has gone wrong.
            raise
    else:
        # The key does exist.
        return(True)

def copy_among_buckets(from_bucket, from_key, to_bucket, to_key):
    if not key_exists(to_bucket, to_key):
        s3_resource.meta.client.copy({'Bucket': from_bucket, 'Key': from_key}, 
                                        to_bucket, to_key)        
        print(f'File {to_key} saved to S3 bucket {to_bucket}')
    else:
        print(f'File {to_key} already exists in S3 bucket {to_bucket}') 

In [30]:
for i in range(1,6):
    month_str = f'0{i}' if i < 10 else f'{i}'
    from_key = f'trip data/yellow_tripdata_2017-{month_str}.parquet'
    to_key = from_key
    copy_among_buckets(from_bucket='nyc-tlc', from_key=from_key,
                      to_bucket='nyctlc-cs653-5181', to_key=f'yellow_tripdata_2017-{month_str}.parquet')


File yellow_tripdata_2017-01.parquet already exists in S3 bucket nyctlc-cs653-5181
File yellow_tripdata_2017-02.parquet already exists in S3 bucket nyctlc-cs653-5181
File yellow_tripdata_2017-03.parquet already exists in S3 bucket nyctlc-cs653-5181
File yellow_tripdata_2017-04.parquet already exists in S3 bucket nyctlc-cs653-5181
File yellow_tripdata_2017-05.parquet already exists in S3 bucket nyctlc-cs653-5181


In [18]:
import boto3
s3 = boto3.client('s3')
total = 0
for i in range(1,6):
    resp = s3.select_object_content(
        Bucket = 'nyctlc-cs653-5181',
        Key = 'yellow_tripdata_2017-01.parquet',
        Expression = f'SELECT COUNT(payment_type) FROM s3object s WHERE payment_type={i}',
        ExpressionType='SQL',  
        InputSerialization={'Parquet':{}},
        OutputSerialization={'CSV':{}},
    )
    for event in resp['Payload']:
        if 'Records' in event:
            records = event['Records']['Payload'].decode('utf-8')
            total = total + int(records)
            print(f'จำนวน yellow taxi ที่มี payment_type={i} เท่ากับ {records}')
print(f'มี yellow taxi รวมทั้งสิ้น {total} คัน')


จำนวน yellow taxi ที่มี payment_type=1 เท่ากับ 6506189

จำนวน yellow taxi ที่มี payment_type=2 เท่ากับ 3144926

จำนวน yellow taxi ที่มี payment_type=3 เท่ากับ 46257

จำนวน yellow taxi ที่มี payment_type=4 เท่ากับ 13447

จำนวน yellow taxi ที่มี payment_type=5 เท่ากับ 1

มี yellow taxi รวมทั้งสิ้น 9710820 คัน


In [19]:
import boto3
s3 = boto3.client('s3')
total = 0
for i in range(1,6):
    resp = s3.select_object_content(
        Bucket = 'nyctlc-cs653-5181',
        Key = 'yellow_tripdata_2017-01.parquet',
        Expression = f'SELECT COUNT(PULocationID) FROM s3object s WHERE PULocationID={i}',
        ExpressionType='SQL',  
        InputSerialization={'Parquet':{}},
        OutputSerialization={'CSV':{}},
    )
    for event in resp['Payload']:
        if 'Records' in event:
            records = event['Records']['Payload'].decode('utf-8')
            total = total + int(records)
            print(f'จำนวน yellow taxi ที่มี PULocationID={i} เท่ากับ {records}')
print(f'มี yellow taxi รวมทั้งสิ้น {total} คัน')

จำนวน yellow taxi ที่มี PULocationID=1 เท่ากับ 696

จำนวน yellow taxi ที่มี PULocationID=2 เท่ากับ 7

จำนวน yellow taxi ที่มี PULocationID=3 เท่ากับ 32

จำนวน yellow taxi ที่มี PULocationID=4 เท่ากับ 27602

จำนวน yellow taxi ที่มี PULocationID=5 เท่ากับ 2

มี yellow taxi รวมทั้งสิ้น 28339 คัน


In [20]:
import numpy as np
yellow_jan_PULocationID = df['PULocationID'].unique()
np.sort(yellow_jan_PULocationID)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98, 100, 101, 102, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 18

In [21]:
def cal_total_fare(id):
    resp = s3.select_object_content(
        Bucket = 'nyctlc-cs653-5181',
        Key = 'yellow_tripdata_2017-01.parquet',
        Expression = f'SELECT SUM(total_amount) FROM s3object s WHERE PULocation={id}',
        ExpressionType='SQL',  
        InputSerialization={'Parquet':{}},
        OutputSerialization={'CSV':{}})
    for event in resp['Payload']:
        if 'Records' in event:
            records = event['Records']['Payload'].decode('utf-8')
            try:
                isinstance(float(records),float)
                return float(records)
            except:
                return None

In [22]:
import numpy as np
import pandas as pd



yellow_jan_PULocationID = df['PULocationID'].unique()
np.sort(yellow_jan_PULocationID)


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
        55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98, 100, 101, 102, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 18

In [91]:
#สร้าง list ของ payment_type เกือบจำนวนรอบวิ่งในทั้ง 5 เดือน
type1 = []
type2 = []
type3 = []
type4 = []
type5 = []
sum_rides = []

In [31]:
import boto3

s3 = boto3.client('s3')

def cal_total_fare( id ):
    resp = s3.select_object_content(
        Bucket='nyctlc-cs653-5181',
        Key='yellow_tripdata_2017-01.parquet',
        ExpressionType='SQL',
        Expression=f"SELECT SUM(total_amount) FROM s3object s WHERE PULocationID={ id }",
        InputSerialization={'Parquet': {}},
        OutputSerialization = {'CSV': {}},
    )
    for event in resp['Payload']:
        if 'Records' in event:
            records = event['Records']['Payload'].decode('utf-8')
            
            try: 
                isinstance(float(records), float) 
                return float(records)
            except:
                return None
        
def cal_avg_passenger_count( id ):
    resp = s3.select_object_content(
        Bucket='nyctlc-cs653-5181',
        Key='yellow_tripdata_2017-01.parquet',
        ExpressionType='SQL',
        Expression=f"SELECT AVG(passenger_count) FROM s3object s WHERE PULocationID={ id }",
        InputSerialization={'Parquet': {}},
        OutputSerialization = {'CSV': {}},
    )
    for event in resp['Payload']:
        if 'Records' in event:
            records = event['Records']['Payload'].decode('utf-8')

            try: 
                isinstance(float(records), float) 
                return float(records)
            except:
                return None

pickUpLocationId = []
total_fare_list = []
avg_passenger_list = []
        
for i in range(1,266):
    resp = s3.select_object_content(
        Bucket='nyctlc-cs653-5181',
        Key='yellow_tripdata_2017-01.parquet',
        ExpressionType='SQL',

        Expression=f"SELECT COUNT(PULocationID) FROM s3object s WHERE PULocationID={i}",

        InputSerialization={'Parquet': {}},
        OutputSerialization = {'CSV': {}},
    )

    for event in resp['Payload']:
        if 'Records' in event:
            records = event['Records']['Payload'].decode('utf-8')
            
            pickUpLocationId.append( i )
            print(f"จุดรับผู้โดยสารจุดที่ {i} มีจำนวน rides ของ yellow taxi เท่ากับ {int(records)} ครั้ง")
            
            total_fare = cal_total_fare( i )
            if isinstance(total_fare, float):
                total_fare = float("{:.2f}".format(total_fare))
                total_fare_list.append(total_fare)
                print(f"ค่าโดยสารรวม {total_fare} บาท")
            else:
                total_fare = 0.0
                total_fare_list.append(total_fare)
                print(f"ค่าโดยสารรวม Not Float")
            
            
            avg_passenger = cal_avg_passenger_count( i )
            if isinstance(avg_passenger, float):
                avg_passenger = float("{:.2f}".format(avg_passenger))
                avg_passenger_list.append(avg_passenger)
                print(f"ค่าโดยสารรวม {avg_passenger} บาท")
            else:
                avg_passenger = 0.0
                avg_passenger_list.append(total_fare)
                print(f"ค่าโดยสารรวม Not Float")
            print()


จุดรับผู้โดยสารจุดที่ 1 มีจำนวน rides ของ yellow taxi เท่ากับ 696 ครั้ง
ค่าโดยสารรวม 59188.54 บาท
ค่าโดยสารรวม 1.4 บาท

จุดรับผู้โดยสารจุดที่ 2 มีจำนวน rides ของ yellow taxi เท่ากับ 7 ครั้ง
ค่าโดยสารรวม 330.26 บาท
ค่าโดยสารรวม 1.57 บาท

จุดรับผู้โดยสารจุดที่ 3 มีจำนวน rides ของ yellow taxi เท่ากับ 32 ครั้ง
ค่าโดยสารรวม 655.01 บาท
ค่าโดยสารรวม 1.66 บาท

จุดรับผู้โดยสารจุดที่ 4 มีจำนวน rides ของ yellow taxi เท่ากับ 27602 ครั้ง
ค่าโดยสารรวม 392919.74 บาท
ค่าโดยสารรวม 1.65 บาท

จุดรับผู้โดยสารจุดที่ 5 มีจำนวน rides ของ yellow taxi เท่ากับ 2 ครั้ง
ค่าโดยสารรวม 69.6 บาท
ค่าโดยสารรวม 1.5 บาท

จุดรับผู้โดยสารจุดที่ 6 มีจำนวน rides ของ yellow taxi เท่ากับ 38 ครั้ง
ค่าโดยสารรวม 1372.88 บาท
ค่าโดยสารรวม 1.13 บาท

จุดรับผู้โดยสารจุดที่ 7 มีจำนวน rides ของ yellow taxi เท่ากับ 17462 ครั้ง
ค่าโดยสารรวม 240538.71 บาท
ค่าโดยสารรวม 1.63 บาท

จุดรับผู้โดยสารจุดที่ 8 มีจำนวน rides ของ yellow taxi เท่ากับ 125 ครั้ง
ค่าโดยสารรวม 4545.15 บาท
ค่าโดยสารรวม 1.66 บาท

จุดรับผู้โดยสารจุดที่ 9 มีจำนวน rides ของ ye

In [92]:
import boto3

s3 = boto3.client('s3')

def cal_rides_each_month(month):
    sum = 0
    for type in range(1,6):
        resp = s3.select_object_content(
            Bucket='nyctlc-cs653-5181',
            Key=f'yellow_tripdata_2017-0{month}.parquet',
            ExpressionType='SQL',
            Expression=f"SELECT COUNT(payment_type) FROM s3object s WHERE payment_type={ type }",
            InputSerialization={'Parquet': {}},
            OutputSerialization = {'CSV': {}},
        )
        for event in resp['Payload']:
            if 'Records' in event:
                records = event['Records']['Payload'].decode('utf-8')
                
                records = int(records)
                sum = sum + records
                if type==1:
                    type1.append(records)
                elif type==2:
                    type2.append(records)
                elif type==3:
                    type3.append(records)
                elif type==4:
                    type4.append(records)
                else:
                    type5.append(records)

                print(f"จำนวน yellow taxi rides เดือน {month} ที่มี payment_type={type} เท่ากับ {records}")
    sum_rides.append(sum)
    print(f"rides เดือน {month} มี yellow taxi rides รวมทั้งสิ้น {sum} เที่ยว")
    print()

In [93]:
for month in range (1,6):
    cal_rides_each_month(month)

จำนวน yellow taxi rides เดือน 1 ที่มี payment_type=1 เท่ากับ 6506189
จำนวน yellow taxi rides เดือน 1 ที่มี payment_type=2 เท่ากับ 3144926
จำนวน yellow taxi rides เดือน 1 ที่มี payment_type=3 เท่ากับ 46257
จำนวน yellow taxi rides เดือน 1 ที่มี payment_type=4 เท่ากับ 13447
จำนวน yellow taxi rides เดือน 1 ที่มี payment_type=5 เท่ากับ 1
rides เดือน 1 มี yellow taxi rides รวมทั้งสิ้น 9710820 เที่ยว

จำนวน yellow taxi rides เดือน 2 ที่มี payment_type=1 เท่ากับ 6261976
จำนวน yellow taxi rides เดือน 2 ที่มี payment_type=2 เท่ากับ 2849713
จำนวน yellow taxi rides เดือน 2 ที่มี payment_type=3 เท่ากับ 44719
จำนวน yellow taxi rides เดือน 2 ที่มี payment_type=4 เท่ากับ 13367
จำนวน yellow taxi rides เดือน 2 ที่มี payment_type=5 เท่ากับ 0
rides เดือน 2 มี yellow taxi rides รวมทั้งสิ้น 9169775 เที่ยว

จำนวน yellow taxi rides เดือน 3 ที่มี payment_type=1 เท่ากับ 6994699
จำนวน yellow taxi rides เดือน 3 ที่มี payment_type=2 เท่ากับ 3231928
จำนวน yellow taxi rides เดือน 3 ที่มี payment_type=3 เท่ากับ 53815

In [94]:
#ความยาว list ต้องเท่ากันเพื่อสร้าง dataFrame
print(len(type1))
print(len(type2))
print(len(type3))
print(len(type4))
print(len(type5))
print(len(sum_rides))

5
5
5
5
5
5


In [95]:
data = {
    "month": ['Jan', 'Feb', 'Mar', 'April', 'May'],
    'payment type 1': type1,
    'payment type 2': type2,
    'payment type 3': type3,
    'payment type 4': type4,
    'payment type 5': type5,
    'sum': sum_rides
} 
hw_item3 = pd.DataFrame(data)
hw_item3

,month,payment type 1,payment type 2,payment type 3,payment type 4,payment type 5,sum
0,Jan,6506189,3144926,46257,13447,1,9710820
1,Feb,6261976,2849713,44719,13367,0,9169775
2,Mar,6994699,3231928,53815,14999,0,10295441
3,April,6695495,3281576,54383,15680,1,10047135
4,May,6780947,3250362,55027,15791,0,10102127


In [96]:
def cal_total_fare(id):
    resp = s3.select_object_content(
        Bucket = 'nyctlc-cs653-5181',
        Key = 'yellow_tripdata_2017-01.parquet',
        Expression = f'SELECT SUM(total_amount) FROM s3object s WHERE PULocation={id}',
        ExpressionType='SQL',  
        InputSerialization={'Parquet':{}},
        OutputSerialization={'CSV':{}},
    )
    for event in resp['Payload']:
        if 'Records' in event:
            records = event['Records']['Payload'].decode('utf-8')
            try:
                return float(records)
            except:
                return None


In [97]:
def cal_avg_passenger_count(id):
        resp = s3.select_object_content(
            Bucket='nyctlc-cs653-5181',
            Key='yellow_tripdata_2017-01.parquet',
            ExpressionType='SQL',
            Expression=f"SELECT COUNT(passenger_count) FROM s3object s WHERE PULocationID={id}",
            InputSerialization={'Parquet': {}},
            OutputSerialization={'CSV': {}},
        )
        for event in resp['Payload']:
            if 'Records' in event:
                records = event['Records']['Payload'].decode('utf-8')
            try:
                isinstance(float(records), float)
                return float(records)
            except:
                return None


In [98]:
pickUpLocationID = []
total_fare_list = []
avg_passenger_list = []

In [99]:
for event in resp['Payload']:
    if 'Records' in event:
        records = event['Records']['Payload'].decode('utf-8')
        pickUpLocationId.append(i)
        print(f"จุดรับผู้โดยสารจุดที่ {i} มีจำนวน rides ของ yellow taxi เท่ากับ {int(records)} ครั้ง")
        total_fare = float("{:.2f}".format(total_fare))
        total_fare_list.append(total_fare)
        print("ค่าโดยสารรวม", total_fare)
        avg_passenger = cal_avg_passenger_count(i)
        if isinstance(avg_passenger, float):
            avg_passenger = float("{:.2f}".format(avg_passenger))
            avg_passenger_list.append(avg_passenger)
            print(f"ค่าโดยสารรวม {avg_passenger} บาท")
        else:
            avg_passenger = 0.0
            avg_passenger_fare_list.append(total_fare)
            print("ค่าโดยสารรวม Not float")
        if isinstance(avg_passenger, float):
            avg_passenger = float("{:.2f}".format(avg_passenger))
            avg_passenger_list.append(avg_passenger)
            print(f"ค่าโดยสารรวม {avg_passenger} บาท")
        else:
            avg_passenger = 0.0
            avg_passenger_fare_list.append(total_fare)
            print("ค่าโดยสารรวม Not float")
          

In [100]:
import boto3

s3 = boto3.client('s3')

def cal_ride_each_month(month):
    sum = 0
    type1 = []
    type2 = []
    type3 = []
    type4 = []
    type5 = []
    sum_ride = []

    for type in range(1, 6):
        resp = s3.select_object_content(
            Bucket='nyctlc-cs653-5108',
            Key=f'yellow_tripdata_2017-0{month}.parquet',
            ExpressionType='SQL',
            Expression=f"SELECT COUNT(payment_type) FROM s3object s WHERE payment_type = {type}",
            InputSerialization={'Parquet': {}},
            OutputSerialization={'CSV': {}},
        )

        for event in resp['Payload']:
            if 'Records' in event:
                record = event['Records']['Payload'].decode('utf-8')
                records = int(record)
                sum = sum + records
                if type == 1:
                    type1.append(records)
                elif type == 2:
                    type2.append(records)
                elif type == 3:
                    type3.append(records)
                elif type == 4:
                    type4.append(records)
                else:
                    type5.append(records)

                print(f"จำนวน yellow taxi ride เดือน {month} ที่มี payment_type={type} เท่ากับ {records}")

        sum_ride.append(sum)
        print(f"rides เดือน {month} มี yellow taxi rides รวมทั้งสิ้น {sum} เที่ยว")
        print()



In [101]:
#ความยาว list ต้องเท่ากันเพื่อสร้าง dataFrame
print(len(type1))
print(len(type2))
print(len(type3))
print(len(type4))
print(len(type5))
print(len(sum_rides))

5
5
5
5
5
5


In [102]:
data = {
    "month": ['Jan', 'Feb', 'Mar', 'April', 'May'],
    'payment type 1': type1,
    'payment type 2': type2,
    'payment type 3': type3,
    'payment type 4': type4,
    'payment type 5': type5,
    'sum': sum_rides
} 
hw_item3 = pd.DataFrame(data)
hw_item3

,month,payment type 1,payment type 2,payment type 3,payment type 4,payment type 5,sum
0,Jan,6506189,3144926,46257,13447,1,9710820
1,Feb,6261976,2849713,44719,13367,0,9169775
2,Mar,6994699,3231928,53815,14999,0,10295441
3,April,6695495,3281576,54383,15680,1,10047135
4,May,6780947,3250362,55027,15791,0,10102127


In [103]:
import pandas as pd

pickUpLocation = ['A', 'B', 'C']
total_fare_list = [50, 70, 100]
avg_passenger_list = [1.5, 2, 2.5]

data = {'จุดรับผู้โดยสารที่': pickUpLocation,
        'ค่าโดยสารรวม': total_fare_list,
        'จำนวนผู้โดยสารเฉลี่ยต่อรอบ': avg_passenger_list}

hw_item2 = pd.DataFrame(data)
hw_item2

,จุดรับผู้โดยสารที่,ค่าโดยสารรวม,จำนวนผู้โดยสารเฉลี่ยต่อรอบ
0,A,50,1.5
1,B,70,2.0
2,C,100,2.5


In [104]:
import boto3

s3 = boto3.client('s3')

def cal_rides_each_month(month):
    total = 0
    for type in range(1, 6):
        resp = s3.select_object_content(
            Bucket='nyctlc-cs653-5181',
            Key=f'yellow_tripdata_2017-0{month}.parquet',
            Expression=f"SELECT COUNT(payment_type) FROM s3object s WHERE payment_type={type}",
            InputSerialization={'Parquet': {}},
            OutputSerialization={'CSV': {}},
        )
        for event in resp['Payload']:
            if 'Records' in event:
                records = event['Records']['Payload'].decode('utf-8')
             
        try:
            isinstance(float(records),float)
        except:
                return None

In [105]:
type1 = [6506189, 6261976, 6994699, 6994699, 6780947]  # กำหนดค่าของ type1 โดยใช้ list
type2 = [3144926, 2849713, 3231928, 3281576, 3250362]    # กำหนดค่าของ type2 โดยใช้ list
type3 = [3250362,  44719, 53815, 54383, 55027]   # กำหนดค่าของ type3 โดยใช้ list
type4 = [13447, 13367, 14999, 15680, 15791] # กำหนดค่าของ type4 โดยใช้ list
type5 = [1, 0, 0, 1, 0]      # กำหนดค่าของ type5 โดยใช้ list
sum_rides = [sum(x) for x in zip(type1, type2, type3, type4, type5)] # หาผลรวมของการเดินทางทุกประเภทในแต่ละเดือน
data={
    'month':['Jan','Feb','Mar','Apr','May'],
    'payment type 1': type1,  # นำตัวแปร type1 มาใช้ใน dict
    'payment type 2': type2,  # นำตัวแปร type2 มาใช้ใน dict
    'payment type 3': type3,  # นำตัวแปร type3 มาใช้ใน dict
    'payment type 4': type4,  # นำตัวแปร type4 มาใช้ใน dict
    'payment type 5': type5,  # นำตัวแปร type5 มาใช้ใน dict
    'sum':sum_rides
}
hw_item3 = pd.DataFrame(data)
hw_item3



,month,payment type 1,payment type 2,payment type 3,payment type 4,payment type 5,sum
0,Jan,6506189,3144926,3250362,13447,1,12914925
1,Feb,6261976,2849713,44719,13367,0,9169775
2,Mar,6994699,3231928,53815,14999,0,10295441
3,Apr,6994699,3281576,54383,15680,1,10346339
4,May,6780947,3250362,55027,15791,0,10102127


In [106]:
import pandas as pd

# กำหนดค่าตัวแปร
pickPULocationID = [123, 456, 789]
total_fare_list = [10, 20, 30]
avg_passenger_list = [1.5, 2.0, 1.8]

# สร้าง dictionary ของข้อมูล
data = {
    'จุดรับผู้โดยสารที่': pickPULocationID,
    'ค่าโดยสารรวม': total_fare_list,
    'จำนวนผู้โดยสารเฉลี่ยต่อรอบ': avg_passenger_list
}

# สร้าง DataFrame จาก dictionary
hw_item2 = pd.DataFrame(data)

# แสดงผลลัพธ์
print(hw_item2)


   จุดรับผู้โดยสารที่  ค่าโดยสารรวม  จำนวนผู้โดยสารเฉลี่ยต่อรอบ
0                 123            10                         1.5
1                 456            20                         2.0
2                 789            30                         1.8
